In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import re

AZURE API

In [2]:
from openai import AzureOpenAI
# gets the API Key from environment variable AZURE_OPENAI_API_KEY
client = AzureOpenAI(
    azure_endpoint="https://team5-chatgpt-4-api.openai.azure.com/",
    api_version = "2023-05-15",  # Use the latest available version
    api_key = "9423fcf02a494b5cbe440c6971903ba7",
)

In [3]:
#read the dataset from the jsonl file
df = pd.read_json('DATASETS\human-eval-modified.jsonl', lines=True)

df.head()




,task_id,prompt,entry_point,canonical_solution,test
0,HumanEval/0,from typing import List\n\n\ndef has_close_ele...,has_close_elements,"for idx, elem in enumerate(numbers):\n ...","\n\nMETADATA = {\n 'author': 'jt',\n 'da..."
1,HumanEval/1,from typing import List\n\n\ndef separate_pare...,separate_paren_groups,result = []\n current_string = []\n ...,"\n\nMETADATA = {\n 'author': 'jt',\n 'da..."
2,HumanEval/2,\n\ndef truncate_number(number: float) -> floa...,truncate_number,return number % 1.0\n,"\n\nMETADATA = {\n 'author': 'jt',\n 'da..."
3,HumanEval/3,from typing import List\n\n\ndef below_zero(op...,below_zero,balance = 0\n\n for op in operations:\n...,"\n\nMETADATA = {\n 'author': 'jt',\n 'da..."
4,HumanEval/4,from typing import List\n\n\ndef mean_absolute...,mean_absolute_deviation,mean = sum(numbers) / len(numbers)\n re...,"\n\nMETADATA = {\n 'author': 'jt',\n 'da..."


Generating the test cases as a list

In [58]:
# chat completion
responses = pd.DataFrame(columns=['taskID','prompt', 'entry_point', 'response'])

taskID = df['task_id'].tolist()
prompts = df['prompt'].tolist()
entry_points = df['entry_point'].tolist()

l = len(df)

tests= 10

for i in range(l):
    ID = taskID[i]
    problem_statement = prompts[i]
    entry_point = entry_points[i]
        
    response = client.chat.completions.create(
        model="gpt4-api",  # The name you used when deploying the model
        messages=[
            {"role": "system", "content": "You are a helpful Python programming assistant. Your objective is to create test inputs for the given problem."},
            {"role": "user", "content": f"""For the given problem statement, create test inputs. The format of the input is a list of inputs.
                                        The problem statement is: {problem_statement}.
                                        The entry point is: {entry_point},
                                        The input list is {tests} inputs representing {tests} test cases. Cover basic test cases and edge cases.
                                        Return only the function calls with each of the test cases, each in a separate line, without any other text."""},
        ]
    )

    #save the response to a variable
    response_text = response.choices[0].message.content
    #create a temp dataframe to store the responses
    temp_df = pd.DataFrame([[ID, problem_statement, entry_point, response_text]], columns=['taskID','prompt', 'entry_point', 'response'])
    #concatenate the temp dataframe to the responses dataframe
    responses = pd.concat([responses, temp_df], ignore_index=True)
    

Parsing the test cases as function calls

In [64]:
def remove_outside_quotes(text):
    # Match quotes outside of parentheses and replace them
    result = re.sub(r'\"(?![^()]*\))', '', text)
    result = re.sub(r"\'(?![^()]*\))", '', result)
    return result


tests = 10


# create a dataframe to store the inputs and to change dynamically (the columns) based on the number of tests
cols = ['task_id'] + [f'test_{i}' for i in range(1, tests+1)]


All = pd.DataFrame(columns=cols)


for row in responses.iterrows():
    #parse the response to get the list of inputs
    inputs = row[1]['response'].split("\n")
    #remove any input that doesn't have the entry point
    inputs = [input for input in inputs if row[1]['entry_point'] in input]

    #remove the quotes from the inputs using the function defined above
    inputs = [remove_outside_quotes(input) for input in inputs]

    #remove any leading or trailing whitespaces
    inputs = [input.strip() for input in inputs]
    #remove commas if they are the last character
    inputs = [input[:-1] if input[-1] == ',' else input for input in inputs]

    # print("\n")
    # print(50*"-")


    id = row[1]['taskID']
    
    # create a temp dataframe to store the inputs
    temp_df = pd.DataFrame([[id, inputs[0], inputs[1], inputs[2], inputs[3], inputs[4], inputs[5], inputs[6], inputs[7], inputs[8], inputs[9]]], 
                           columns=cols)

    #concatenate the temp dataframe to the All dataframe
    All = pd.concat([All, temp_df], ignore_index=True)


In [65]:
All

,task_id,test_1,test_2,test_3,test_4,test_5,test_6,test_7,test_8,test_9,test_10
0,HumanEval/0,"has_close_elements([], 1.0)","has_close_elements([1.0], 0.5)","has_close_elements([1.0, 2.0, 3.0], 1.5)","has_close_elements([1.0, 3.0, 2.1], 0.2)","has_close_elements([0.1, 0.15, 0.3], 0.05)","has_close_elements([10.0, 20.0, 20.1], 0.15)","has_close_elements([1.5, 2.0, 3.0, 1.55], 0.06)","has_close_elements([-1.0, -2.0, -1.99], 0.01)","has_close_elements([1.23456, 1.23458, 2.34567]...","has_close_elements([5.5, 5.7, 5.9], 0.22)"
1,HumanEval/1,separate_paren_groups(((()))'),separate_paren_groups(() (()) ((()))'),separate_paren_groups((( )( ))'),separate_paren_groups((( )( )) (( ))'),separate_paren_groups((( ( )(( )) ))'),separate_paren_groups(( )( )'),separate_paren_groups(''),separate_paren_groups((((())))'),separate_paren_groups(()()()'),separate_paren_groups((())()((()))')
2,HumanEval/2,truncate_number(0.75),truncate_number(10.01),truncate_number(5.0),truncate_number(1.9999),truncate_number(999.999),truncate_number(0.0001),truncate_number(123.123),truncate_number(0.5),truncate_number(100.89),truncate_number(6000.000001)
3,HumanEval/3,"below_zero([100, -50, -30])",below_zero([-1]),"below_zero([1, -2, 3, -4, 5, -10])","below_zero([0, 0, 0, 0])","below_zero([-100, 50, 60])","below_zero([5, 15, -20, 5, -6])","below_zero([15, -10, -7, 2])","below_zero([1, 2, 3, -6])","below_zero([10, -1, -1, -1, -1, -1, -1, -1, -1...","below_zero([7, 14, -30, 9])"
4,HumanEval/4,"mean_absolute_deviation([1.0, 2.0, 3.0, 4.0, 5...","mean_absolute_deviation([1.1, 1.1, 1.1, 1.1])","mean_absolute_deviation([-5.0, 0.0, 5.0, 10.0])",mean_absolute_deviation([100]),mean_absolute_deviation([]),"mean_absolute_deviation([10.5, 10.5, 10.5, 10.5])","mean_absolute_deviation([-2.5, -1.0, 0.0, 1.0,...",mean_absolute_deviation([0.33]),"mean_absolute_deviation([0.0001, 100000])","mean_absolute_deviation([2.5, -3.2, 4.8, -2.5,..."
...,...,...,...,...,...,...,...,...,...,...,...
159,HumanEval/159,"eat(0, 0, 0)","eat(5, 0, 20)","eat(0, 10, 5)","eat(1000, 1000, 1000)","eat(500, 300, 200)","eat(250, 750, 500)","eat(5, 500, 5)","eat(900, 80, 0)","eat(600, 700, 800)","eat(0, 0, 1000)"
160,HumanEval/160,"do_algebra(['+'], [2, 3])","do_algebra(['-', '+', '*'], [10, 5, 2, 3])","do_algebra(['*', '+', '//'], [1, 2, 3, 4])","do_algebra(['**', '-'], [2, 3, 4])","do_algebra(['+', '*', '-', '//', '**'], [2, 3,...","do_algebra(['//', '*', '+'], [100, 10, 5, 7])","do_algebra(['-'], [10, 4])","do_algebra(['**', '//', '+', '*'], [3, 2, 4, 5...","do_algebra(['*'], [7, 3])","do_algebra(['+', '-', '*', '**', '//'], [1, 4,..."
161,HumanEval/161,"solve(""1234"")","solve(""ab"")","solve(""#a@C"")","solve(""abcdEFGHIJ"")","solve("""")","solve(""1111"")","solve(""123abcDEF"")","solve(""@#$%^"")","solve(""argUING1234"")","solve(""spaces are cool"")"
162,HumanEval/162,string_to_md5('Hello world'),string_to_md5(''),string_to_md5('password123'),string_to_md5('123'),string_to_md5('string_to_md5'),string_to_md5('Example with spaces'),string_to_md5('special@#chars&*!$'),string_to_md5('<html><body>This is a test</bod...,string_to_md5('a'),string_to_md5('Non-English text like: текст')


In [66]:
#merge all with df to get the final dataframe
final_df = pd.merge(df, All, on = 'task_id', how='left')
final_df

,task_id,prompt,entry_point,canonical_solution,test,test_1,test_2,test_3,test_4,test_5,test_6,test_7,test_8,test_9,test_10
0,HumanEval/0,from typing import List\n\n\ndef has_close_ele...,has_close_elements,"for idx, elem in enumerate(numbers):\n ...","\n\nMETADATA = {\n 'author': 'jt',\n 'da...","has_close_elements([], 1.0)","has_close_elements([1.0], 0.5)","has_close_elements([1.0, 2.0, 3.0], 1.5)","has_close_elements([1.0, 3.0, 2.1], 0.2)","has_close_elements([0.1, 0.15, 0.3], 0.05)","has_close_elements([10.0, 20.0, 20.1], 0.15)","has_close_elements([1.5, 2.0, 3.0, 1.55], 0.06)","has_close_elements([-1.0, -2.0, -1.99], 0.01)","has_close_elements([1.23456, 1.23458, 2.34567]...","has_close_elements([5.5, 5.7, 5.9], 0.22)"
1,HumanEval/1,from typing import List\n\n\ndef separate_pare...,separate_paren_groups,result = []\n current_string = []\n ...,"\n\nMETADATA = {\n 'author': 'jt',\n 'da...",separate_paren_groups(((()))'),separate_paren_groups(() (()) ((()))'),separate_paren_groups((( )( ))'),separate_paren_groups((( )( )) (( ))'),separate_paren_groups((( ( )(( )) ))'),separate_paren_groups(( )( )'),separate_paren_groups(''),separate_paren_groups((((())))'),separate_paren_groups(()()()'),separate_paren_groups((())()((()))')
2,HumanEval/2,\n\ndef truncate_number(number: float) -> floa...,truncate_number,return number % 1.0\n,"\n\nMETADATA = {\n 'author': 'jt',\n 'da...",truncate_number(0.75),truncate_number(10.01),truncate_number(5.0),truncate_number(1.9999),truncate_number(999.999),truncate_number(0.0001),truncate_number(123.123),truncate_number(0.5),truncate_number(100.89),truncate_number(6000.000001)
3,HumanEval/3,from typing import List\n\n\ndef below_zero(op...,below_zero,balance = 0\n\n for op in operations:\n...,"\n\nMETADATA = {\n 'author': 'jt',\n 'da...","below_zero([100, -50, -30])",below_zero([-1]),"below_zero([1, -2, 3, -4, 5, -10])","below_zero([0, 0, 0, 0])","below_zero([-100, 50, 60])","below_zero([5, 15, -20, 5, -6])","below_zero([15, -10, -7, 2])","below_zero([1, 2, 3, -6])","below_zero([10, -1, -1, -1, -1, -1, -1, -1, -1...","below_zero([7, 14, -30, 9])"
4,HumanEval/4,from typing import List\n\n\ndef mean_absolute...,mean_absolute_deviation,mean = sum(numbers) / len(numbers)\n re...,"\n\nMETADATA = {\n 'author': 'jt',\n 'da...","mean_absolute_deviation([1.0, 2.0, 3.0, 4.0, 5...","mean_absolute_deviation([1.1, 1.1, 1.1, 1.1])","mean_absolute_deviation([-5.0, 0.0, 5.0, 10.0])",mean_absolute_deviation([100]),mean_absolute_deviation([]),"mean_absolute_deviation([10.5, 10.5, 10.5, 10.5])","mean_absolute_deviation([-2.5, -1.0, 0.0, 1.0,...",mean_absolute_deviation([0.33]),"mean_absolute_deviation([0.0001, 100000])","mean_absolute_deviation([2.5, -3.2, 4.8, -2.5,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,HumanEval/159,"\ndef eat(number, need, remaining):\n """"""\n...",eat,if(need <= remaining):\n return [ n...,def check(candidate):\n total_tests_xyz = 6...,"eat(0, 0, 0)","eat(5, 0, 20)","eat(0, 10, 5)","eat(1000, 1000, 1000)","eat(500, 300, 200)","eat(250, 750, 500)","eat(5, 500, 5)","eat(900, 80, 0)","eat(600, 700, 800)","eat(0, 0, 1000)"
160,HumanEval/160,"\ndef do_algebra(operator, operand):\n """"""\...",do_algebra,expression = str(operand[0])\n for oprt...,def check(candidate):\n total_tests_xyz = 3...,"do_algebra(['+'], [2, 3])","do_algebra(['-', '+', '*'], [10, 5, 2, 3])","do_algebra(['*', '+', '//'], [1, 2, 3, 4])","do_algebra(['**', '-'], [2, 3, 4])","do_algebra(['+', '*', '-', '//', '**'], [2, 3,...","do_algebra(['//', '*', '+'], [100, 10, 5, 7])","do_algebra(['-'], [10, 4])","do_algebra(['**', '//', '+', '*'], [3, 2, 4, 5...","do_algebra(['*'], [7, 3])","do_algebra(['+', '-', '*', '**', '//'], [1, 4,..."
161,HumanEval/161,"\ndef solve(s):\n """"""You are given a string...",solve,flg = 0\n idx = 0\n new_str = list(s...,def check(candidate):\n total_tests_xyz = 8...,"solve(""1234"")","solve(""ab"")","solve(""#a@C"")","solve(""abcdEFGHIJ"")","solve("""")","solve(""1111"")","solv

In [67]:
final_df.to_csv('DATASETs\human-eval-modified-with-tests.csv', index=False)

#export the final dataframe to a jsonl file
final_df.to_json('DATASETS\human-eval-modified-with-tests.jsonl', orient='records', lines=True)